#Import Dataset

In [28]:
import pandas as pd
import numpy as np
import seaborn as sns

In [29]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.decomposition import FactorAnalysis
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import confusion_matrix as cm
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split, StratifiedKFold
from sklearn.feature_selection import RFE
import statsmodels.api as sm
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import matthews_corrcoef
from sklearn.preprocessing import LabelEncoder

In [30]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [31]:
import os

root_dir = "/content/gdrive/MyDrive/Colab Notebooks/2-1 패턴인식"

# Checking if our specified directory exists
os.path.exists(root_dir)

True

In [32]:
#train
data_path = os.path.join(root_dir, "train.csv")
df_train = pd.read_csv(data_path)

#test
data_path = os.path.join(root_dir, "test.csv")
df_test = pd.read_csv(data_path)

In [33]:
df_train

,id,age,job,marital,education,default,housing,loan,contact,month,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,12224,46,admin.,married,high.school,unknown,no,no,telephone,jul,...,3,999,0,nonexistent,1.4,93.918,-42.7,4.966,5228.1,no
1,36387,32,admin.,married,university.degree,no,yes,yes,cellular,jun,...,1,999,0,nonexistent,-2.9,92.963,-40.8,1.262,5076.2,yes
2,22703,36,technician,single,university.degree,no,no,no,cellular,aug,...,5,999,0,nonexistent,1.4,93.444,-36.1,4.964,5228.1,no
3,32742,30,services,single,high.school,no,yes,yes,cellular,may,...,1,999,2,failure,-1.8,92.893,-46.2,1.299,5099.1,no
4,40547,76,retired,divorced,basic.4y,no,no,no,cellular,aug,...,1,999,1,failure,-1.7,94.027,-38.3,0.886,4991.6,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32945,24109,37,admin.,married,university.degree,no,yes,no,telephone,nov,...,1,6,1,success,-0.1,93.200,-42.0,4.286,5195.8,no
32946,14149,56,unknown,single,basic.9y,no,no,yes,cellular,jul,...,5,999,0,nonexistent,1.4,93.918,-42.7,4.962,5228.1,no
32947,12379,55,admin.,married,high.school,no,yes,no,cellular,jul,...,3,999,0,nonexistent,1.4,93.918,-42.7,4.960,5228.1,no
32948,10138,34,services,married,high.school,no,no,no,telephone,jun,...,1,999,0,nonexistent,1.4,94.465,-41.8,4.958,5228.1,no


In [34]:
# Encode the Categorical Variables.

df_train["month"] = df_train["month"].replace({"jul": 7, "jun": 6, "aug": 8, "may": 5, "oct" : 10, 'sep': 9, "nov": 11, "apr": 4, "mar":3, "dec": 12})

df_train["year"] = df_train["cons.price.idx"]
df_train["year"] = df_train["cons.price.idx"].replace({93.918 : 2008, 92.963 : 2009, 93.444 : 2008, 92.893 : 2009,
                                                       94.027 : 2010, 92.431 : 2009, 94.199 : 2010, 94.465 : 2008,
                                                       92.649 : 2009, 93.994 : 2008, 93.075 : 2009, 93.2 : 2008,
                                                       94.601 : 2010, 92.469 : 2009, 92.201 : 2009, 93.369 : 2010,
                                                       93.876 : 2010, 92.713 : 2009, 93.798 : 2008, 92.843 : 2009,
                                                       92.379 : 2009, 94.055 : 2010, 94.215 : 2010, 93.749 : 2010,
                                                       94.767 : 2010, 92.756 : 2008})

In [35]:
df_train['job'].replace('unknown', df_train['job'].mode().iloc[0], inplace=True)

In [36]:
df_train['loan'].replace('unknown', df_train['loan'].mode().iloc[0], inplace=True)

In [37]:
df_train['marital'].replace('unknown', df_train['marital'].mode().iloc[0], inplace=True)

In [38]:
conti_var_new =df_train.describe().columns

In [39]:
df_train = df_train.drop(labels=["euribor3m", "nr.employed"], axis=1)

In [40]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32950 entries, 0 to 32949
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              32950 non-null  int64  
 1   age             32950 non-null  int64  
 2   job             32950 non-null  object 
 3   marital         32950 non-null  object 
 4   education       32950 non-null  object 
 5   default         32950 non-null  object 
 6   housing         32950 non-null  object 
 7   loan            32950 non-null  object 
 8   contact         32950 non-null  object 
 9   month           32950 non-null  int64  
 10  day_of_week     32950 non-null  object 
 11  campaign        32950 non-null  int64  
 12  pdays           32950 non-null  int64  
 13  previous        32950 non-null  int64  
 14  poutcome        32950 non-null  object 
 15  emp.var.rate    32950 non-null  float64
 16  cons.price.idx  32950 non-null  float64
 17  cons.conf.idx   32950 non-null 

#make Dummy variables

In [41]:
columns = df_train.select_dtypes(include=[object]).columns
df = pd.concat([df_train, pd.get_dummies(df_train[columns])], axis=1)
df = df.drop(['job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week','poutcome', 'y'], axis =1)
print("\n","Describe Dataset_v2","\n")
print(df.info(),"\n \n \n")


 Describe Dataset_v2 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32950 entries, 0 to 32949
Data columns (total 51 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   id                             32950 non-null  int64  
 1   age                            32950 non-null  int64  
 2   campaign                       32950 non-null  int64  
 3   pdays                          32950 non-null  int64  
 4   previous                       32950 non-null  int64  
 5   emp.var.rate                   32950 non-null  float64
 6   cons.price.idx                 32950 non-null  float64
 7   cons.conf.idx                  32950 non-null  float64
 8   year                           32950 non-null  float64
 9   job_admin.                     32950 non-null  bool   
 10  job_blue-collar                32950 non-null  bool   
 11  job_entrepreneur               32950 non-null  bool   
 12  job_housemaid         

In [42]:
df = df.drop('default_yes', axis = 1)

In [43]:
df.columns

Index(['id', 'age', 'campaign', 'pdays', 'previous', 'emp.var.rate',
       'cons.price.idx', 'cons.conf.idx', 'year', 'job_admin.',
       'job_blue-collar', 'job_entrepreneur', 'job_housemaid',
       'job_management', 'job_retired', 'job_self-employed', 'job_services',
       'job_student', 'job_technician', 'job_unemployed', 'marital_divorced',
       'marital_married', 'marital_single', 'education_basic.4y',
       'education_basic.6y', 'education_basic.9y', 'education_high.school',
       'education_illiterate', 'education_professional.course',
       'education_university.degree', 'education_unknown', 'default_no',
       'default_unknown', 'housing_no', 'housing_unknown', 'housing_yes',
       'loan_no', 'loan_yes', 'contact_cellular', 'contact_telephone',
       'day_of_week_fri', 'day_of_week_mon', 'day_of_week_thu',
       'day_of_week_tue', 'day_of_week_wed', 'poutcome_failure',
       'poutcome_nonexistent', 'poutcome_success', 'y_no', 'y_yes'],
      dtype='object')

#Scaling, Train-Test Split, Downsampling

In [44]:
min_max_scaler=preprocessing.MinMaxScaler()
data_scaled=pd.DataFrame(min_max_scaler.fit_transform(df),columns=df.columns)

In [45]:
y = data_scaled.y_yes
data_scaled = data_scaled.drop(['y_yes','y_no'], axis = 1)

In [46]:
df = df.drop(['y_yes','y_no'], axis = 1)

In [47]:
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.under_sampling import NeighbourhoodCleaningRule

In [48]:
from imblearn.under_sampling import NeighbourhoodCleaningRule

# NeighbourhoodCleaningRule을 사용하여 데이터 샘플링
X_samp, y_samp = NeighbourhoodCleaningRule(kind_sel="mode", n_neighbors=7).fit_resample(data_scaled, y)

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X_samp, y_samp, shuffle=True, test_size=0.2, random_state=42)

#Training & Validation

In [50]:
!pip install catboost
from catboost import CatBoostClassifier, Pool, cv
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold

In [ ]:
# CatBoost 모델 정의
model = CatBoostClassifier()

param_grid = {'iterations': [200],'depth': [5], 'learning_rate': [0.1],'l2_leaf_reg': [5]}

from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='f1', cv=loo, verbose=1, n_jobs=-1 )

# model training
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_score =  grid_search.best_score_

print(best_params)
print(best_score)

best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

print(classification_report(y_test, y_pred))

Fitting 17024 folds for each of 1 candidates, totalling 17024 fits


In [ ]:
## CatBoost 모델 정의
#model = CatBoostClassifier()

#param_grid = {'iterations': [300],'depth': [5],'learning_rate': [0.01, 0.05, 0.1],'l2_leaf_reg': [3, 5, 7]}

#grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='f1', cv=5, verbose=1, n_jobs=-1 )

## model training
#grid_search.fit(X_train, y_train)

#best_params = grid_search.best_params_
#best_score =  grid_search.best_score_

#print(best_params)
#print(best_score)

#best_model = grid_search.best_estimator_
#y_pred = best_model.predict(X_test)

#print(classification_report(y_test, y_pred))

#Evaluation

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

y_prob = best_model.predict_proba(X_test)
print("Estimated probs:", y_prob[:10])

y_cls = best_model.predict(X_test)
print("Estimated classes:", y_cls[:10])
print()

In [ ]:
print("Accuracy:", accuracy_score(y_test, y_cls))
print("F1:", f1_score(y_test, y_cls))
print("ROC AUC:", roc_auc_score(y_test, y_prob[:, 1]))

#Test with answer set

In [ ]:
import os

root_dir = "/content/gdrive/MyDrive/Colab Notebooks/"

# Checking if our specified directory exists
os.path.exists(root_dir)

In [ ]:
# Paths to the downloaded files
data_path = os.path.join(root_dir, "merged_with_answer (1).csv")

# Load data
df_test = pd.read_csv(data_path)

In [ ]:
df_test.head()

In [ ]:
df_test

In [ ]:
df_test["month"] = df_test["month"].replace({"jul": 7, "jun": 6, "aug": 8, "may": 5, "oct" : 10, 'sep': 9, "nov": 11, "apr": 4, "mar":3, "dec": 12})

df_test["year"] = df_test["cons.price.idx"]
df_test["year"] = df_test["cons.price.idx"].replace({93.918 : 2008, 92.963 : 2009, 93.444 : 2008, 92.893 : 2009,
                                                       94.027 : 2010, 92.431 : 2009, 94.199 : 2010, 94.465 : 2008,
                                                       92.649 : 2009, 93.994 : 2008, 93.075 : 2009, 93.2 : 2008,
                                                       94.601 : 2010, 92.469 : 2009, 92.201 : 2009, 93.369 : 2010,
                                                       93.876 : 2010, 92.713 : 2009, 93.798 : 2008, 92.843 : 2009,
                                                       92.379 : 2009, 94.055 : 2010, 94.215 : 2010, 93.749 : 2010,
                                                       94.767 : 2010, 92.756 : 2008})

In [ ]:
df_test['job'].replace('unknown', df_test['job'].mode().iloc[0], inplace=True)

In [ ]:
df_test['loan'].replace('unknown', df_test['loan'].mode().iloc[0], inplace=True)

In [ ]:
df_test['marital'].replace('unknown', df_test['marital'].mode().iloc[0], inplace=True)

In [ ]:
df_test = df_test.drop(["key", "Unnamed: 0"], axis = 1)

In [ ]:
columns = df_test.select_dtypes(include=[object]).columns
df1 = pd.concat([df_test, pd.get_dummies(df_test[columns])], axis=1)
df1 = df1.drop(['job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week','poutcome', 'y'], axis =1)
print("\n","Describe Dataset_v2","\n")
print(df1.info(),"\n \n \n")

In [ ]:
df1.head()

In [ ]:
min_max_scaler=preprocessing.MinMaxScaler()
data_scaled1=pd.DataFrame(min_max_scaler.fit_transform(df1),columns=df1.columns)

In [ ]:
yt = data_scaled1.y_yes
xt = data_scaled1.drop(['y_yes','y_no'], axis = 1)

In [ ]:
xt.head()

In [ ]:
xt.columns

In [ ]:
df.columns

In [ ]:
xt = xt[['id', 'age', 'campaign', 'pdays', 'previous', 'emp.var.rate',
       'cons.price.idx', 'cons.conf.idx', 'year',
       'job_admin.', 'job_blue-collar', 'job_entrepreneur', 'job_housemaid',
       'job_management', 'job_retired', 'job_self-employed', 'job_services',
       'job_student', 'job_technician', 'job_unemployed',
       'marital_divorced', 'marital_married', 'marital_single', 'education_basic.4y', 'education_basic.6y',
       'education_basic.9y', 'education_high.school', 'education_illiterate',
       'education_professional.course', 'education_university.degree',
       'education_unknown', 'default_no', 'default_unknown',
       'housing_no', 'housing_unknown', 'housing_yes', 'loan_no',
       'loan_yes', 'contact_cellular', 'contact_telephone',
       'day_of_week_fri', 'day_of_week_mon', 'day_of_week_thu',
       'day_of_week_tue', 'day_of_week_wed', 'poutcome_failure',
       'poutcome_nonexistent', 'poutcome_success']]

In [ ]:
!pip install catboost
from catboost import CatBoostClassifier, Pool, cv
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold

In [ ]:
y_final = best_model.predict(xt)

# F1 점수 계산
f1 = f1_score(yt, y_final)
print("F1 Score:", f1)


#Evaluation

In [ ]:
y_final = pd.Series(y_final)



In [ ]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

y_prob = best_model.predict_proba(xt)
print("Estimated probs:", y_prob[:10])

y_cls = best_model.predict(xt)
print("Estimated classes:", y_cls[:10])
print()

In [ ]:
print("Accuracy:", accuracy_score(yt, y_cls))
print("F1:", f1_score(yt, y_cls))
print("ROC AUC:", roc_auc_score(yt, y_prob[:, 1]))

In [ ]:
from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(yt, y_cls)
conf_matrix_df = pd.DataFrame(
    conf_matrix,
    columns=["Predicted yes", "Predicted no"],
    index=["Actual yes", "Actual no"]
)
print(conf_matrix_df)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(yt, y_cls))